In [1]:
import os
import pandas as pd
import numpy as np

# 1. Recopilación de información de archivos
Usamos la función os.walk para que este genere los nombres en un arbol de directorios 

In [4]:
root_dir = '.\\assets'

routes = {}

for sub_dir, dirs, files in os.walk(root_dir):
    for file in files:
        file_path = sub_dir + os.sep + file

        # Prevent open temporal files
        if file_path.endswith(".xls") and not file.startswith("~$"):
            routes[file_path] = file.replace(".xls", "")



routes

{'.\\assets\\VT051101.xls': 'VT051101',
 '.\\assets\\VT051102.xls': 'VT051102',
 '.\\assets\\VT051103.xls': 'VT051103',
 '.\\assets\\VT051104.xls': 'VT051104',
 '.\\assets\\VT051105.xls': 'VT051105'}

In [5]:
dataframes = {}

for path, file_name in routes.items():
    print(file_name)
    dataframes[path] = pd.read_excel(path)


# Mostramos solo el primero para mejor visualización
list(dataframes.values())[0]

VT051101
VT051102
VT051103
VT051104
VT051105


,Fecha 1,Fecha 2,Hora,Sala,Caja,N_correlativo,N_boleta,Ingreso_Egreso,Ean13,Cantidad,Gramaje,Monto
0,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780295050254,32,0,3168
1,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780295050255,10,0,990
2,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,550706,0,502,801
3,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,550706,0,538,859
4,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780281000337,3,0,882
...,...,...,...,...,...,...,...,...,...,...,...,...
28900,2005-11-01,2005-12-30,20:57:00,9,21,323,264878,N,645158,0,1000,2498
28901,2005-11-01,2005-12-30,20:59:00,9,21,324,264879,N,645160,0,1000,658
28902,2005-11-01,2005-12-30,20:59:00,9,21,324,264879,N,645791,0,1000,499
28903,2005-11-01,2005-12-30,21:02:00,9,21,325,264880,N,645158,0,1000,2498


# 2. Pregunta a resolver

¿Cuál es la caja top de mayor a menor en cantidad de ventas por día? 
Podemos hacer el top del tamaño que queramos. 

###  2.1) Primer paso

Primero crearemos boxes_by_fyle para contabilizar la cantidad de veces que se repitio una caja en el dia, por documento

In [6]:
boxes_by_file = {}

for path, dataframe in dataframes.items():
    boxes_entries = dataframe["Caja"].to_numpy()
    boxes = list(set(boxes_entries.flatten()))
    for box in boxes:
        if boxes_by_file.get(box) is None:
            boxes_by_file[box] = {}

        box_amount = dataframe["Caja"][dataframe["Caja"] == box].count()

        boxes_by_file[box].update({routes[path]: box_amount})


#Mostramos solo el primero para verificar que funcione y tener una visualizacion mejor
list(boxes_by_file.items())[0]

(1,
 {'VT051101': 301,
  'VT051102': 773,
  'VT051103': 539,
  'VT051104': 760,
  'VT051105': 533})

### 2.2) Segundo paso
Ya que la cantidad de ventas por caja es la misma que la cantidad de veces que aparece en el archivo, utilizamos la informacion adquerida anteriormente ocupando boxes_entries

In [7]:
sells_by_file = {}

for path, dataframe in dataframes.items():
    sells_by_file[path] = {}
    boxes_entries = dataframe["Caja"].to_numpy()
    boxes = list(set(boxes_entries.flatten()))
    for box in boxes:
        box_amount = dataframe["Caja"][dataframe["Caja"] == box].count()

        sells_by_file[path].update({box: box_amount})


#Mostramos solo el primero para verificar que funcione y tener una visualizacion mejor
list(sells_by_file.items())[0]

('.\\assets\\VT051101.xls',
 {1: 301,
  2: 1299,
  3: 128,
  4: 1790,
  5: 1497,
  6: 2470,
  7: 2390,
  9: 2004,
  11: 2090,
  12: 2284,
  13: 2173,
  14: 2041,
  15: 2027,
  16: 1022,
  17: 2234,
  18: 1585,
  20: 1014,
  21: 556})

### 2.3) Tercer paso


In [8]:
top_sells_by_file = {}

for path, sells in sells_by_file.items():
    sorted_sells = sorted(sells.items(), key=lambda box: box[1], reverse=True)
    top_three = sorted_sells[:] # Here can be selected how many top sell boxes are
    top_sells_by_file[path] = top_three



list(top_sells_by_file.items())[0]

('.\\assets\\VT051101.xls',
 [(6, 2470),
  (7, 2390),
  (12, 2284),
  (17, 2234),
  (13, 2173),
  (11, 2090),
  (14, 2041),
  (15, 2027),
  (9, 2004),
  (4, 1790),
  (18, 1585),
  (5, 1497),
  (2, 1299),
  (16, 1022),
  (20, 1014),
  (21, 556),
  (1, 301),
  (3, 128)])

### 2.4) Cuarto paso
Creamos una tabla con los valores entregados en top_sells_by_file y asignamos nombres a los indices de nuestra tabla.


In [9]:
name_file = "File"
name_box = "Box"
name_sells = "Sells"
name_top_in_day = "Top in day"

top_sells_dict = {name_file: [], name_box: [], name_sells: [], name_top_in_day: []}

for path, top_sells in top_sells_by_file.items():
    for i, data in enumerate(top_sells, start=1):
        box, sell = data
        top_sells_dict[name_file].append(routes[path])
        top_sells_dict[name_box].append(box)
        top_sells_dict[name_sells].append(sell)
        top_sells_dict[name_top_in_day].append(i)



list(top_sells_dict.keys())

['File', 'Box', 'Sells', 'Top in day']

In [10]:
top_sells_dataframe = pd.DataFrame(top_sells_dict)

top_sells_dataframe[top_sells_dataframe[name_top_in_day].isin([2])]

,File,Box,Sells,Top in day
1,VT051101,7,2390,2
19,VT051102,12,2885,2
40,VT051103,14,2893,2
62,VT051104,7,3004,2
84,VT051105,12,2928,2


# 3. Limpieza de datos
Limpiaremos los datos que estan erroneos, por ejemplo; tenemos montos con valores como 0000-354, al igual que la cantidad y gramaje. 
Eliminaremos esos datos

In [11]:
consolidated = pd.concat(dataframes.values()).reset_index().drop('index',1)

consolidated

,Fecha 1,Fecha 2,Hora,Sala,Caja,N_correlativo,N_boleta,Ingreso_Egreso,Ean13,Cantidad,Gramaje,Monto
0,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780295050254,32,0,3168
1,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780295050255,10,0,990
2,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,550706,0,502,801
3,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,550706,0,538,859
4,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780281000337,3,0,882
...,...,...,...,...,...,...,...,...,...,...,...,...
177619,2005-11-05,2005-12-30,22:11:00,9,21,394,266477,N,645479,0,2000,1596
177620,2005-11-05,2005-12-30,18:15:00,9,89,1,0,S,645588,0,2000,00000-998
177621,2005-11-05,2005-12-30,18:30:00,9,89,3,0,S,780494500142,00000-1,0,0000-1129
177622,2005-11-05,2005-12-30,19:56:00,9,89,5,0,S,780492300710,00000-1,0,0000-1478


In [12]:
int_filter = lambda entry: isinstance(entry, int) and entry >= 0
cleaned_consolidated = consolidated.copy()

### Limpieza 1
Limpiamos los valores de la columna cantidad

In [13]:
cleaned_consolidated = cleaned_consolidated[cleaned_consolidated["Cantidad"].apply(int_filter)]

cleaned_consolidated

,Fecha 1,Fecha 2,Hora,Sala,Caja,N_correlativo,N_boleta,Ingreso_Egreso,Ean13,Cantidad,Gramaje,Monto
0,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780295050254,32,0,3168
1,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780295050255,10,0,990
2,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,550706,0,502,801
3,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,550706,0,538,859
4,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780281000337,3,0,882
...,...,...,...,...,...,...,...,...,...,...,...,...
177616,2005-11-05,2005-12-30,22:03:00,9,21,392,266475,N,645160,0,1000,658
177617,2005-11-05,2005-12-30,22:04:00,9,21,393,266476,N,645892,1,0,500
177618,2005-11-05,2005-12-30,22:04:00,9,21,393,266476,N,645892,1,0,500
177619,2005-11-05,2005-12-30,22:11:00,9,21,394,266477,N,645479,0,2000,1596


### Limpieza 2
Limpiamos los valores de la columna monto

In [14]:
cleaned_consolidated = cleaned_consolidated[cleaned_consolidated["Monto"].apply(int_filter)]

cleaned_consolidated

,Fecha 1,Fecha 2,Hora,Sala,Caja,N_correlativo,N_boleta,Ingreso_Egreso,Ean13,Cantidad,Gramaje,Monto
0,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780295050254,32,0,3168
1,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780295050255,10,0,990
2,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,550706,0,502,801
3,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,550706,0,538,859
4,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780281000337,3,0,882
...,...,...,...,...,...,...,...,...,...,...,...,...
177615,2005-11-05,2005-12-30,21:58:00,9,21,391,266474,N,645158,0,1000,2498
177616,2005-11-05,2005-12-30,22:03:00,9,21,392,266475,N,645160,0,1000,658
177617,2005-11-05,2005-12-30,22:04:00,9,21,393,266476,N,645892,1,0,500
177618,2005-11-05,2005-12-30,22:04:00,9,21,393,266476,N,645892,1,0,500


### Limpieza 3
Limpiamos los valores de la columna gramaje.

In [15]:
cleaned_consolidated = cleaned_consolidated[cleaned_consolidated["Gramaje"].apply(int_filter)]

cleaned_consolidated

,Fecha 1,Fecha 2,Hora,Sala,Caja,N_correlativo,N_boleta,Ingreso_Egreso,Ean13,Cantidad,Gramaje,Monto
0,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780295050254,32,0,3168
1,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780295050255,10,0,990
2,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,550706,0,502,801
3,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,550706,0,538,859
4,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780281000337,3,0,882
...,...,...,...,...,...,...,...,...,...,...,...,...
177615,2005-11-05,2005-12-30,21:58:00,9,21,391,266474,N,645158,0,1000,2498
177616,2005-11-05,2005-12-30,22:03:00,9,21,392,266475,N,645160,0,1000,658
177617,2005-11-05,2005-12-30,22:04:00,9,21,393,266476,N,645892,1,0,500
177618,2005-11-05,2005-12-30,22:04:00,9,21,393,266476,N,645892,1,0,500


### Limpieza 4
Limpiamos los valores de la columna Ean13


In [16]:
cleaned_consolidated = cleaned_consolidated[cleaned_consolidated["Ean13"].apply(int_filter)]

cleaned_consolidated

,Fecha 1,Fecha 2,Hora,Sala,Caja,N_correlativo,N_boleta,Ingreso_Egreso,Ean13,Cantidad,Gramaje,Monto
0,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780295050254,32,0,3168
1,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780295050255,10,0,990
2,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,550706,0,502,801
3,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,550706,0,538,859
4,2005-11-01,2005-12-30,09:51:00,9,1,1,0,N,780281000337,3,0,882
...,...,...,...,...,...,...,...,...,...,...,...,...
177615,2005-11-05,2005-12-30,21:58:00,9,21,391,266474,N,645158,0,1000,2498
177616,2005-11-05,2005-12-30,22:03:00,9,21,392,266475,N,645160,0,1000,658
177617,2005-11-05,2005-12-30,22:04:00,9,21,393,266476,N,645892,1,0,500
177618,2005-11-05,2005-12-30,22:04:00,9,21,393,266476,N,645892,1,0,500


#### ¿Cuántos datos fueron limpiados?

In [18]:
how_many_cleaned = len(consolidated.index) - len(cleaned_consolidated.index)

print('\nSe limpiaron', how_many_cleaned,'datos.')



Se limpiaron 1120 datos.


In [26]:
eans_by_file ={}

for path, dataframe in dataframes.items():
    eans_by_file[path] = {}
    eans_entries = dataframe["Ean13"].to_numpy()
    eans = list(set(eans_entries.flatten()))
    for ean in eans:
        ean_amount = dataframe["Ean13"][dataframe["Ean13"] == ean].count()

        eans_by_file[path].update({ean: ean_amount})


#Mostramos solo el primero para verificar que funcione y tener una visualizacion mejor
list(eans_by_file.keys())[0]

'.\\assets\\VT051101.xls'

In [32]:
eans_in_file = {}

for path, dataframe in dataframes.items():
    ean_entries = dataframe["Ean13"].to_numpy()
    eans = list(set(ean_entries.flatten()))
    for ean in eans:
        if eans_in_file.get(ean) is None:
            eans_in_file[ean] = {}

        eans_amount = dataframe["Ean13"][dataframe["Ean13"] == ean].count()

        eans_in_file[ean].update({routes[path]: eans_amount})


#Mostramos solo el primero para verificar que funcione y tener una visualizacion mejor
list(eans_in_file.items())[0]

(780222300160, {'VT051101': 2, 'VT051105': 1})

In [43]:
eans_in_total = {}

for ean, total_by_day in eans_in_file.items():
    eans_in_total[ean] = sum(total_by_day.values())
   
list(eans_in_total.items())[0]

(780222300160, 3)

In [48]:
sorted_eans = sorted(eans_in_total.items(), key=lambda ean: ean[1], reverse=True)

sorted_eans[:3]

[(720001, 5761), (720005, 2365), (780150500511, 1856)]

In [63]:
box_by_ean = {}

for ean in eans_in_file.keys():
    box_by_ean[ean] = {}
    for box in boxes_by_file.keys():
        box_by_ean[ean][box] = consolidated['Caja'][(consolidated['Ean13'] == ean) & (consolidated['Caja'] == box)].count()
        


list(box_by_ean.items())[0]

(780222300160,
 {1: 0,
  2: 0,
  3: 0,
  4: 0,
  5: 0,
  6: 1,
  7: 1,
  9: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0,
  15: 0,
  16: 0,
  17: 0,
  18: 0,
  20: 1,
  21: 0,
  8: 0,
  10: 0,
  89: 0,
  19: 0})

In [68]:
box_by_ean_top = []

for ean, total_in_box in box_by_ean.items():
    for box, total in total_in_box.items():
        box_by_ean_top.append([ean, box, total])
    

sorter = lambda row: row[2]
box_by_ean_top.sort(key=sorter, reverse=True)

box_by_ean_top[:5]

[[720001, 17, 500],
 [720001, 20, 476],
 [720001, 12, 462],
 [645158, 21, 453],
 [720001, 14, 401]]

### Resultado 1
En la tabla se agrupan productos (código de barra), caja y el resultado de cuantas veces ese producto paso por la caja, ordenandolos de mayor a menor.

In [69]:
df = pd.DataFrame(box_by_ean_top)

df

,0,1,2
0,720001,17,500
1,720001,20,476
2,720001,12,462
3,645158,21,453
4,720001,14,401
...,...,...,...
188381,780583600100,21,0
188382,780583600100,8,0
188383,780583600100,10,0
188384,780583600100,89,0
